<a href="https://colab.research.google.com/github/ThongData/SQL-Python-Project/blob/main/Danny_Ma_Pizza_Challenge_SQL_vs_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import Image, display

In [2]:
Image(url='https://github.com/ThongData/SQL-Python-Project/blob/main/Pizza_Runner/Image/Pizza%20Runner%20Schema.png?raw=true', width=800)

**Case Study background**

Imagine you are a data analyst for a pizza company with the above data models.
usually, pizza stores got delivery service which is directly affecting the quality of the pizza, and customer satisfaction as customers always want to taste the hot tasty pizzas in the short time after they place the orders.

Moreover, it is also equally important to know the customer preferences and behaviors through the history data of what they orders, thereby improving the business performances.

Today, you are working on a project to clean up the data, and answer some of the questions from CEO who is really keen on knowing more about the status quo of the business

# I. Preparing the environment and connection to MYSQL database
This part is mainly for importing the necessary libraries which I have used in this case study as well as establish a connection to the free cloude database hosting (Postgresql)

In [70]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import warnings
import re
from sqlalchemy import create_engine
warnings.simplefilter('ignore')

In [4]:
params={"host":"ep-curly-glitter-63573327.ap-southeast-1.aws.neon.tech",
 "database":"neondb",
 "user":"thongnm1992",
 "password":"zr5gTOnNWb8X"}

In [5]:
engine=create_engine("postgresql://{}:{}@{}:5432/{}".format(params['user'],params['password'],params['host'],params['database']))
def connect_to_db():
    conn = psycopg2.connect(host=params["host"],
    database=params["database"],
    user=params["user"],
    password=params["password"])
    return conn
conn = connect_to_db()
cur = conn.cursor()

# II. Convert tables from MYSQL to dataframe for running anlaysis by pandas

As I am going to solve this case study by using both Python and SQL, I imported the data from SQL database to this notebook as dataframe for later analysis.



In [6]:
Customer_orders = pd.read_sql('select * from customer_orders',conn)
Customer_orders.head(5)

,order_id,customer_id,pizza_id,exclusions,extras,order_time
0,1,101,1,,,2020-01-01 18:05:02
1,2,101,1,,,2020-01-01 19:00:52
2,3,102,1,,,2020-01-02 23:51:23
3,3,102,2,,None,2020-01-02 23:51:23
4,4,103,1,4,,2020-01-04 13:23:46


In [7]:
runner_orders = pd.read_sql('select * from runner_orders',conn)
runner_orders

,order_id,runner_id,pickup_time,distance,duration,cancellation
0,1,1,2020-01-01 18:15:34,20km,32 minutes,
1,2,1,2020-01-01 19:10:54,20km,27 minutes,
2,3,1,2020-01-03 00:12:37,13.4km,20 mins,None
3,4,2,2020-01-04 13:53:03,23.4,40,None
4,5,3,2020-01-08 21:10:57,10,15,None
5,6,3,null,null,null,Restaurant Cancellation
6,7,2,2020-01-08 21:30:45,25km,25mins,null
7,8,2,2020-01-10 00:15:02,23.4 km,15 minute,null
8,9,2,null,null,null,Customer Cancellation
9,10,1,2020-01-11 18:50:20,10km,10minutes,null


In [8]:
runners = pd.read_sql('select * from runners', conn)
runners

,runner_id,registration_date
0,1,2021-01-01
1,2,2021-01-03
2,3,2021-01-08
3,4,2021-01-15


In [9]:
pizza_names = pd.read_sql('select * from pizza_names', conn)
pizza_names

,pizza_id,pizza_name
0,1,Meatlovers
1,2,Vegetarian


In [10]:
pizza_recipes = pd.read_sql('select * from pizza_recipes', conn)
pizza_recipes

,pizza_id,toppings
0,1,"1, 2, 3, 4, 5, 6, 8, 10"
1,2,"4, 6, 7, 9, 11, 12"


In [301]:
pizza_toppings = pd.read_sql('select * from pizza_toppings', conn)
pizza_toppings

,topping_id,topping_name
0,1,Bacon
1,2,BBQ Sauce
2,3,Beef
3,4,Cheese
4,5,Chicken
5,6,Mushrooms
6,7,Onions
7,8,Pepperoni
8,9,Peppers
9,10,Salami


# III. Data transformation

Data is, in the real business wolrd, always a mess and you are undoutably required to transform it.
This section, I demonstrate using both SQL and Python to do the same cleaning tasks.
If you are either SQL or Python person, you can refer to python codes to see how different it is between SQL and Python.

## ① Runner_orders Table: Cancellation column

The column cancellation of Runner_orders got a mixed data of blank, None, Null and cancellation. Therefore, we need to standardize it by the following logic:

1. None, null and blank → null
2. the rest: keep it as it as

we will clean the data and save it as a different dataframe named 'cleaned_runner_orders'.
Next step is to import the cleaned table to Mysql and save it as a temporay table for further analysis.

### 1.1 Using Python and loading data back to database

In [12]:
runner_orders

,order_id,runner_id,pickup_time,distance,duration,cancellation
0,1,1,2020-01-01 18:15:34,20km,32 minutes,
1,2,1,2020-01-01 19:10:54,20km,27 minutes,
2,3,1,2020-01-03 00:12:37,13.4km,20 mins,None
3,4,2,2020-01-04 13:53:03,23.4,40,None
4,5,3,2020-01-08 21:10:57,10,15,None
5,6,3,null,null,null,Restaurant Cancellation
6,7,2,2020-01-08 21:30:45,25km,25mins,null
7,8,2,2020-01-10 00:15:02,23.4 km,15 minute,null
8,9,2,null,null,null,Customer Cancellation
9,10,1,2020-01-11 18:50:20,10km,10minutes,null


In [13]:
cleaned_runner_orders = runner_orders.copy()
cleaned_runner_orders['cancellation'] = cleaned_runner_orders['cancellation'].replace(['','None','null'],[None,None,None])
cleaned_runner_orders

,order_id,runner_id,pickup_time,distance,duration,cancellation
0,1,1,2020-01-01 18:15:34,20km,32 minutes,None
1,2,1,2020-01-01 19:10:54,20km,27 minutes,None
2,3,1,2020-01-03 00:12:37,13.4km,20 mins,None
3,4,2,2020-01-04 13:53:03,23.4,40,None
4,5,3,2020-01-08 21:10:57,10,15,None
5,6,3,null,null,null,Restaurant Cancellation
6,7,2,2020-01-08 21:30:45,25km,25mins,None
7,8,2,2020-01-10 00:15:02,23.4 km,15 minute,None
8,9,2,null,null,null,Customer Cancellation
9,10,1,2020-01-11 18:50:20,10km,10minutes,None


If you are more comfortable with cleaning the data with python, you can directly load the data into database by dataframe as below

In [14]:
cleaned_runner_orders.to_sql('cleaned_runner_orders', con=engine, if_exists ='replace')

10

###1.2 Using SQL to clean the data

In [15]:
query ="""
drop table if exists cleaned_runner_orders;

create table cleaned_runner_orders as
select
r.order_id,
r.runner_id,
r.pickup_time,
r.distance,
r.duration,
case
	when r.cancellation = '' then null
    when r.cancellation = 'null' then null
    when r.cancellation ='NULL' then null
    when r.cancellation is null then null
    else 'cancellation'
end as cancellation
from runner_orders r;

"""

In [16]:
with connect_to_db() as conn:
    with conn.cursor() as curs:
        curs.execute(query)
        conn.commit()

In [17]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute("select * from cleaned_runner_orders")
        data= curs.fetchall()

# Approach 1
dict_result = []
for row in data:
    dict_result.append(dict(row))
pd.DataFrame(dict_result)

#Approach 2: list comprehesion
pd.DataFrame([dict(x) for x in data])

#Approach 3:
pd.DataFrame([i.copy() for i in data])

,order_id,runner_id,pickup_time,distance,duration,cancellation
0,1,1,2020-01-01 18:15:34,20km,32 minutes,None
1,2,1,2020-01-01 19:10:54,20km,27 minutes,None
2,3,1,2020-01-03 00:12:37,13.4km,20 mins,None
3,4,2,2020-01-04 13:53:03,23.4,40,None
4,5,3,2020-01-08 21:10:57,10,15,None
5,6,3,null,null,null,cancellation
6,7,2,2020-01-08 21:30:45,25km,25mins,None
7,8,2,2020-01-10 00:15:02,23.4 km,15 minute,None
8,9,2,null,null,null,cancellation
9,10,1,2020-01-11 18:50:20,10km,10minutes,None


## ② Customer_order table: values in extras and exclusions column are delimited by comma

### 2.1 Using Python and loading data back to database

In [18]:
customer_orders_temp = Customer_orders.copy()

In [19]:
customer_orders_temp  = customer_orders_temp.assign(row_num=customer_orders_temp.index + 1)

In [20]:
customer_orders_temp = customer_orders_temp.apply(lambda x: x.str.split(',') if x.name in ['exclusions','extras'] else x)

In [21]:
customer_orders_temp = customer_orders_temp.explode('exclusions').explode('extras').replace(['','null'],[None,None])

In [22]:
customer_orders_temp.to_sql('customer_orders_temp', con=engine, if_exists ='replace')

18

In [23]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute("select * from customer_orders_temp")
        data= curs.fetchall()
pd.DataFrame([i.copy() for i in data])

,index,order_id,customer_id,pizza_id,exclusions,extras,order_time,row_num
0,0,1,101,1,None,None,2020-01-01 18:05:02,1
1,1,2,101,1,None,None,2020-01-01 19:00:52,2
2,2,3,102,1,None,None,2020-01-02 23:51:23,3
3,3,3,102,2,None,None,2020-01-02 23:51:23,4
4,4,4,103,1,4,None,2020-01-04 13:23:46,5
5,5,4,103,1,4,None,2020-01-04 13:23:46,6
6,6,4,103,2,4,None,2020-01-04 13:23:46,7
7,7,5,104,1,None,1,2020-01-08 21:00:29,8
8,8,6,101,2,None,None,2020-01-08 21:03:13,9
9,9,7,105,2,None,1,2020-01-08 21:20:29,10


### 2.2 Using SQL to clean data

In [24]:
query ="""
drop table if exists customer_orders_temp;

Create table customer_orders_temp as(
with cte as (
    select row_number() over() AS row_num,
    order_id,
    customer_id,
    pizza_id,
    case
			when exclusions ='' then null
            when exclusions ='null' then null
            else exclusions
    end as exclusions,
    case
			when extras ='' then null
            when extras ='null' then null
            else extras
    end as extras,
    order_time
    from customer_orders)

SELECT
c.row_num,
c.order_id,
c.customer_id,
c.pizza_id,
k.exclusions,
v.extras,
c.order_time

FROM
cte c

left JOIN LATERAL unnest(string_to_array(c.exclusions,',')) k(exclusions) ON true
left JOIN LATERAL unnest(string_to_array(c.extras,',')) v(extras) ON true);

"""

In [25]:
with connect_to_db() as conn:
    with conn.cursor() as curs:
        curs.execute(query)
        conn.commit()

In [26]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute("select * from customer_orders_temp")
        data= curs.fetchall()
pd.DataFrame([i.copy() for i in data])

,row_num,order_id,customer_id,pizza_id,exclusions,extras,order_time
0,1,1,101,1,None,None,2020-01-01 18:05:02
1,2,2,101,1,None,None,2020-01-01 19:00:52
2,3,3,102,1,None,None,2020-01-02 23:51:23
3,4,3,102,2,None,None,2020-01-02 23:51:23
4,5,4,103,1,4,None,2020-01-04 13:23:46
5,6,4,103,1,4,None,2020-01-04 13:23:46
6,7,4,103,2,4,None,2020-01-04 13:23:46
7,8,5,104,1,None,1,2020-01-08 21:00:29
8,9,6,101,2,None,None,2020-01-08 21:03:13
9,10,7,105,2,None,1,2020-01-08 21:20:29


# IV. Analyze Data

Game on! without further ado, let's solve the puzzle with both SQL and Python.
It is really a bumpy journey for me as I need to do lots of googling and recap for what I leanrt. It turned out to be a really enjoyable learning session.

## ① Section 1: Pizza Metrics

### 1.1 How many pizzas were ordered?

**SQL**

In [27]:
query = """
select count(pizza_id) from customer_orders
"""

In [28]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,count
0,14


**Python**

In [29]:
Customer_orders['order_id'].count()

14

### 1.2. How many unique customer orders were made?

**SQL**

In [30]:
query="""
select count(distinct order_id) from customer_orders
"""

In [31]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,count
0,10


**Python**

In [32]:
Customer_orders['order_id'].drop_duplicates().count()

10

### 1.3. How many successful orders were delivered by each runner?

**SQL**

In [33]:
query="""
select count(r.order_id) from cleaned_runner_orders r
where r.cancellation is null
"""

In [34]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,count
0,8


**Python**



In [35]:
cleaned_runner_orders.query('cancellation.isnull()')['order_id'].count()

8

### 1.4. How many of each type of pizza was delivered?

**SQL**

In [36]:
query ="""
select c.pizza_id, count(c.pizza_id) from customer_orders c
left join cleaned_runner_orders r using (order_id)
where r.cancellation is null
group by c.pizza_id
"""

In [37]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,pizza_id,count
0,1,9
1,2,3


**Python**

In [38]:
(Customer_orders
 .merge(cleaned_runner_orders[['cancellation','order_id']], on="order_id")
 .query('cancellation.isnull()')
 .groupby('pizza_id')
 .agg(Count=('pizza_id', 'count'))).merge(pizza_names, on ='pizza_id')[["pizza_name","Count"]]

,pizza_name,Count
0,Meatlovers,9
1,Vegetarian,3


### 1.5. How many Vegetarian and Meatlovers were ordered by each customer?

**SQL**

In [39]:
query ="""
select customer_id, p.pizza_name, count(p.pizza_name) from customer_orders c
join pizza_names p using (pizza_id)
group by c.customer_id, p.pizza_name
order by c.customer_id
"""

In [40]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,customer_id,pizza_name,count
0,101,Meatlovers,2
1,101,Vegetarian,1
2,102,Meatlovers,2
3,102,Vegetarian,1
4,103,Meatlovers,3
5,103,Vegetarian,1
6,104,Meatlovers,3
7,105,Vegetarian,1


**Python**

In [41]:
Customer_orders.merge(pizza_names, on ='pizza_id').groupby(["customer_id","pizza_name"]).agg(Count=('pizza_name','count'))

Count
customer_id pizza_name       
101         Meatlovers      2
            Vegetarian      1
102         Meatlovers      2
            Vegetarian      1
103         Meatlovers      3
            Vegetarian      1
104         Meatlovers      3
105         Vegetarian      1

### 1.6. What was the maximum number of pizzas delivered in a single order?

**SQL**

In [42]:
query ="""
select count(pizza_id) as count_ from customer_orders c
join cleaned_runner_orders r using(order_id)
where r.cancellation is null
group by c.order_id
order by count_ desc
limit 1
"""

In [43]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,count_
0,3


**Python**

In [44]:
(Customer_orders
 .merge(cleaned_runner_orders, on ="order_id")
 .query('cancellation.isnull()')
 .groupby('order_id')
 .agg(Count=('pizza_id',"count"))
 .sort_values(by="Count",ascending=False)).iloc[:1]

,Count
order_id,
4,3


### 1.7. For each customer, how many delivered pizzas had at least 1 change and how many had no changes?


**SQL**

In [45]:
query="""
with cte as (
    select row_number() over() AS row_num,
    order_id,
    customer_id,
    pizza_id,
    case
			when exclusions ='' then null
            when exclusions ='null' then null
            else exclusions
    end as exclusions,
    case
			when extras ='' then null
            when extras ='null' then null
            else extras
    end as extras,
    order_time
    from customer_orders)

select c.customer_id,
sum( case when (exclusions is not null or extras is not null) then 1 else 0 end) change,
sum( case when exclusions is null and extras is null then 1 else 0 end) no_change from cte c
inner join cleaned_runner_orders r using(order_id)
where r.cancellation is null
group by c.customer_id
"""

In [46]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,customer_id,change,no_change
0,101,0,2
1,102,0,3
2,103,3,0
3,104,2,1
4,105,1,0


**Python**

In [47]:
def func(x):
    if x["exclusions"] == None and x["extras"] == None:
        return 1
    else:
        return 0

def func2(x):
    if x["exclusions"] != None or x["extras"] != None:
        return 1
    else:
        return 0

Customer_orders[['exclusions','extras']]=Customer_orders[['exclusions','extras']].replace(['','null'],[None,None])

(Customer_orders
 .merge(cleaned_runner_orders, on='order_id')
 .query('cancellation.isnull()')
 .assign(no_change = lambda x:x.apply(func, axis=1))
 .assign(change = lambda x:x.apply(func2, axis=1))
 .groupby('customer_id')
 .agg({'change':'sum','no_change':'sum'})
 .reset_index()
 )

,customer_id,change,no_change
0,101,0,2
1,102,0,3
2,103,3,0
3,104,2,1
4,105,1,0


### 1.8. How many pizzas were delivered that had both exclusions and extras?

**SQL**

In [48]:
query ="""
with cte as (
    select row_number() over() AS row_num,
    order_id,
    customer_id,
    pizza_id,
    case
			when exclusions ='' then null
            when exclusions ='null' then null
            else exclusions
    end as exclusions,
    case
			when extras ='' then null
            when extras ='null' then null
            else extras
    end as extras,
    order_time
    from customer_orders)

select
sum( case when (exclusions is not null and extras is not null) then 1 else 0 end) both_exclusions_extras from cte c
inner join cleaned_runner_orders r using(order_id)
where r.cancellation is null
"""

In [49]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,both_exclusions_extras
0,1


**Python**

In [50]:
def func(x):
    if x["exclusions"] != None and x["extras"] != None:
        return 1
    else:
        return 0

Customer_orders[['exclusions','extras']]=Customer_orders[['exclusions','extras']].replace(['','null'],[None,None])

(Customer_orders
 .merge(cleaned_runner_orders, on='order_id')
 .query('cancellation.isnull()')
 .assign(both_exclusions_extras = lambda x:x.apply(func, axis=1))["both_exclusions_extras"]
 .sum()
 )

1

### 1.9. What was the total volume of pizzas ordered for each hour of the day?

**SQL**

In [51]:
query ="""
select extract(hour from order_time) as Hour_of_day,
count(order_id) as Order_Num,
round(100*count(order_id)/sum(count(order_id)) over(),2) as volume_of_pizza_deliverd
from customer_orders
Group by Hour_of_day
Order by Hour_of_day
"""

In [52]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,hour_of_day,order_num,volume_of_pizza_deliverd
0,11,1,7.14
1,13,3,21.43
2,18,3,21.43
3,19,1,7.14
4,21,3,21.43
5,23,3,21.43


**Python**

In [53]:
# Approach 1

(Customer_orders
 .assign(Hour_of_day=lambda x: x["order_time"].dt.hour)
 .groupby('Hour_of_day',as_index=False)
 .agg(order_num=('order_id','count'))
 .assign(vol=lambda x: round(100*x["order_num"]/x["order_num"].sum(),2)))

# #Approach 2
# (Customer_orders
#  .assign(Hour_of_day=lambda x: x["order_time"].dt.hour)
#  .groupby('Hour_of_day')
#  .agg(order_num=('order_id','count'))).assign(vol=lambda x:round(100*x["order_num"]/x["order_num"].agg("sum"),2)).reset_index()

,Hour_of_day,order_num,vol
0,11,1,7.14
1,13,3,21.43
2,18,3,21.43
3,19,1,7.14
4,21,3,21.43
5,23,3,21.43


### 1.10. What was the volume of orders for each day of the week?

**SQL**

In [54]:
query ="""
select
To_Char(order_time, 'Day') as Day_of_Week,
count(order_id) as orders,
round(100*count(order_id)/sum(count(order_id)) over(),2) as volume_of_pizza_deliverd
from customer_orders
group by Day_of_Week
order by Day_of_Week desc
"""

In [55]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,day_of_week,orders,volume_of_pizza_deliverd
0,Wednesday,5,35.71
1,Thursday,3,21.43
2,Saturday,5,35.71
3,Friday,1,7.14


**Python**

In [56]:
(Customer_orders
.assign(day_of_the_week=lambda x: x["order_time"].dt.day_name())
.groupby('day_of_the_week')
.agg(order_num=('order_id','count'))
.assign(vol=lambda x: round(100*x["order_num"]/x["order_num"].sum(),2))
.sort_values(by=['day_of_the_week'],ascending = False)
)

,order_num,vol
day_of_the_week,,
Wednesday,5,35.71
Thursday,3,21.43
Saturday,5,35.71
Friday,1,7.14


## ② Section 2: Runner and Customer Experience

### 2.1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)

**SQL**

・WW	Week number of year (1-53) (the first week starts on the first day of the year)

・IW	Week number of ISO 8601 week-numbering year (01-53; the first Thursday of the year is in week 1)

In [57]:
query ="""
SELECT to_char(registration_date::date, 'ww')::int as week,
count(runner_id)
from runners
group by week
order by week
"""

In [58]:
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,week,count
0,1,2
1,2,1
2,3,1


**Python**

In [94]:
(runners
.assign(week_num=lambda x: x["registration_date"].astype('datetime64[ns]').dt.strftime('%W').astype('int')+1)
.groupby('week_num')
.agg(count=('runner_id','count'))
.reset_index()
)

,week_num,count
0,1,2
1,2,1
2,3,1


### 2.2.  What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

**SQL**

In [60]:
query ="""
select avg(EXTRACT(EPOCH FROM (r.pickup_time::timestamp- c.order_time))/60)
from customer_orders c
left join runner_orders r using(order_id)
where r.cancellation is null
"""

In [61]:
# query="""select extract(epoch from '02/22/2022 7:50'::timestamp)"""
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,avg
0,23.4638888888888889


**Python**

In [62]:
(Customer_orders
 .merge(runner_orders, on='order_id')
 .query('cancellation.isnull()')
 .assign(time_to_pickup=lambda x: (x['pickup_time'].astype('datetime64[ns]')-x["order_time"].astype('datetime64[ns]')).astype('timedelta64[s]')/60)
)["time_to_pickup"].mean()

23.46388888888889

### 2.3.  Is there any relationship between the number of pizzas and how long the order takes to prepare?

**SQL**

In [63]:
query ="""
with cte as (
select
count(order_id) pizza_num,
(EXTRACT(EPOCH FROM (r.pickup_time::timestamp- c.order_time))/60::integer) prep_time
from customer_orders c
left join runner_orders r using(order_id)
where r.cancellation is null
group by order_id, r.pickup_time, c.order_time)

select pizza_num,
round(avg(prep_time),2)
from cte c
where prep_time>0
group by pizza_num
"""

In [64]:
# query="""select extract(epoch from '02/22/2022 7:50'::timestamp)"""
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,pizza_num,round
0,1,10.47
1,2,21.23
2,3,29.28


**Python**

In [65]:
(Customer_orders
 .merge(runner_orders, on="order_id")
 .query('cancellation.isnull()')
 .assign(Prep_time=lambda x: round((x['pickup_time'].astype('datetime64[ns]')-x["order_time"].astype('datetime64[ns]')).astype('timedelta64[s]')/60,2))
 .groupby('order_id')
 .agg(pizza_num= ('order_id','count'),
      Avg_prep_time=('Prep_time',"mean"))
 .sort_values(by='pizza_num')
 )

,pizza_num,Avg_prep_time
order_id,,
5,1,10.47
3,2,21.23
4,3,29.28


### 2.4. What was the average distance travelled for each customer?

**SQL**

In [66]:
query="""
select
c.customer_id,
round(cast(avg(substring(r.distance, '^[0-9]*\.?[0-9]+')::float) as numeric),2) from customer_orders c
join cleaned_runner_orders r using(order_id)
where r.cancellation is null
group by customer_id
"""

In [67]:
# query="""select extract(epoch from '02/22/2022 7:50'::timestamp)"""
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,customer_id,round
0,101,20.00
1,102,16.73
2,103,23.40
3,104,10.00
4,105,25.00


**Python**

In [104]:
(Customer_orders
.merge(cleaned_runner_orders, on = 'order_id')
.query('cancellation.isnull()')
.assign(Avg_distance = lambda x: x["distance"].str.split('[a-z]').str[0].replace('',0).astype(float))
.groupby('customer_id')
.agg(Avg_distance=('Avg_distance',lambda x: np.round(np.mean(x),2)))
.reset_index()
)

,customer_id,Avg_distance
0,101,20.00
1,102,16.73
2,103,23.40
3,104,10.00
4,105,25.00


### 2.5. What was the difference between the longest and shortest delivery times for all orders?

**SQL**

In [136]:
query ="""
with cte as(
select substring(duration, '^[0-9]*\.?[0-9]+') as duration from cleaned_runner_orders)
select max(duration)::int - min(duration)::int as difference from cte
"""

In [137]:
# query="""select extract(epoch from '02/22/2022 7:50'::timestamp)"""
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,difference
0,30


**Python**

In [142]:
(cleaned_runner_orders
 .query('cancellation.isnull()')
 .assign(duration=lambda x: x['duration'].str.split('[a-z]').str[0].astype(int))
 .agg(difference=('duration',lambda x: max(x)-min(x)))
 )

,duration
difference,30


### 2.6. What was the average speed for each runner for each delivery and do you notice any trend for these values?


**SQL**

In [192]:
query="""
select
runner_id,
order_id,
substring(duration, '^[0-9]*\.?[0-9]+')::int as duration,
substring(distance, '^[0-9]*\.?[0-9]+')::float as distance,
round(cast(substring(distance, '^[0-9]*\.?[0-9]+')::float/(substring(duration, '^[0-9]*\.?[0-9]+')::float/60) as numeric),2) as Avg_Speed
from cleaned_runner_orders
where cancellation is null
group by runner_id,order_id, duration, distance
"""

In [193]:
# query="""select extract(epoch from '02/22/2022 7:50'::timestamp)"""
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,runner_id,order_id,duration,distance,avg_speed
0,1,1,32,20.0,37.50
1,1,2,27,20.0,44.44
2,1,3,20,13.4,40.20
3,1,10,10,10.0,60.00
4,2,4,40,23.4,35.10
5,2,7,25,25.0,60.00
6,2,8,15,23.4,93.60
7,3,5,15,10.0,40.00


**Python**

In [200]:
(cleaned_runner_orders
 .query('cancellation.isnull()')
 .assign(duration=lambda x: x['duration'].str.split('[a-z]').str[0].astype(int))
 .assign(distance=lambda x: x['distance'].str.split('[a-z]').str[0].astype(float))
 .assign(avg_speed=lambda x: round(x["distance"]/(x["duration"]/60),2))
 .groupby(['runner_id','order_id'])
 .agg(avg_speed=('avg_speed',lambda x: np.round(np.mean(x))))
 )

avg_speed
runner_id order_id           
1         1              38.0
          2              44.0
          3              40.0
          10             60.0
2         4              35.0
          7              60.0
          8              94.0
3         5              40.0

### 2.7. What is the successful delivery percentage for each runner?

**SQL**

In [270]:
query="""
select
runner_id,
round(cast(sum(case when cancellation is null then 1 else 0 end)::float/count(order_id)::float*100 as numeric),1)
from cleaned_runner_orders
group by runner_id;
"""

In [271]:
# query="""select extract(epoch from '02/22/2022 7:50'::timestamp)"""
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,runner_id,round
0,3,50.0
1,2,75.0
2,1,100.0


**Python**

In [296]:
(cleaned_runner_orders
 .assign(success=lambda x: [1 if i == None else 0 for i in x["cancellation"]])
 .groupby('runner_id')
 .agg(Pct=('success',lambda x: np.round(100*sum(x)/x.count(),2)))
 .reset_index()
)

,runner_id,Pct
0,1,100.0
1,2,75.0
2,3,50.0


## ③ Ingredient Optimisation

### 3.1 What are the standard ingredients for each pizza?

**SQL**

In [376]:
query="""
with cte as(

select pizza_id, k.toppings::int from pizza_recipes r

left JOIN LATERAL unnest(string_to_array(toppings,',')) k(toppings) ON true)

select pizza_id, topping_name from cte c

inner join pizza_toppings p on p.topping_id = c.toppings
"""

In [322]:
# query="""select extract(epoch from '02/22/2022 7:50'::timestamp)"""
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,pizza_id,topping_name
0,1,Bacon
1,1,BBQ Sauce
2,1,Beef
3,2,Cheese
4,1,Cheese
5,1,Chicken
6,1,Mushrooms
7,2,Mushrooms
8,2,Onions
9,1,Pepperoni


**Python**

In [341]:
((pizza_recipes
 .assign(toppings=lambda x: x["toppings"].str.split(','))
 )
.explode("toppings").astype(int)
.merge(pizza_toppings, left_on="toppings", right_on="topping_id"))[["pizza_id","topping_name"]]

,pizza_id,topping_name
0,1,Bacon
1,1,BBQ Sauce
2,1,Beef
3,1,Cheese
4,2,Cheese
5,1,Chicken
6,1,Mushrooms
7,2,Mushrooms
8,1,Pepperoni
9,1,Salami


### 3.2 What was the most commonly added extra?

**SQL**

In [375]:
query="""
select p.topping_id, p.topping_name, count(extras) as extras_count from customer_orders_temp c

inner join pizza_toppings p on c.extras::int = p.topping_id::int

inner join cleaned_runner_orders r using(order_id)

where c.extras is not null and r.cancellation is null

group by topping_id, p.topping_name

order by extras_count desc

limit 1
"""

In [374]:
# query="""select extract(epoch from '02/22/2022 7:50'::timestamp)"""
with connect_to_db() as conn:
    with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as curs:
        curs.execute(query)
        data= curs.fetchall()
#Approach 3:
pd.DataFrame([i.copy() for i in data])

,topping_id,topping_name,extras_count
0,1,Bacon,4


**Python**

In [388]:
customer_orders_temp["extras"]=customer_orders_temp["extras"].astype('Int64')
(customer_orders_temp[pd.notnull(customer_orders_temp.extras)]
 .merge(pizza_toppings, left_on='extras', right_on='topping_id')
 .groupby('topping_name')
 .agg(extras_count=('extras','count'))
 .sort_values(by='extras_count', ascending=False)
 .iloc[:1,:]
 .reset_index()
)

,topping_name,extras_count
0,Bacon,5
